In [1]:
import re
import json
import random
from pprint import pprint
from ast import literal_eval
from collections import Counter

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
random.seed(2021)
np.random.seed(2021)

In [3]:
import os
os.listdir(".")

['af_widgets_of_interest.csv',
 'extended_project_list.csv',
 'projects_chosen_by_patrice.csv',
 'exportdata.csv',
 'pillars_subpillars_matching.csv',
 'af_exportables.csv',
 'analysis_frameworks.csv',
 '.ipynb_checkpoints',
 'entries_of_projects_chosen_by_patrice.csv',
 'clean_projects.csv',
 'projects_chosen_by_patrice.ipynb',
 'user_names.csv',
 'queries_v.04.2.txt',
 'prepare_clean_entry_list.ipynb']

In [4]:
exportdata = pd.read_csv("exportdata.csv")
entries = pd.read_csv("entries_of_projects_chosen_by_patrice.csv")
af_widgets = pd.read_csv("af_widgets_of_interest.csv")
projects = pd.read_csv("projects_chosen_by_patrice.csv")
exportables = pd.read_csv("af_exportables.csv")
##
afs = pd.read_csv("analysis_frameworks.csv")
users = pd.read_csv("user_names.csv")
##
matching = pd.read_csv("pillars_subpillars_matching.csv")

In [5]:
af_widgets["properties"] = af_widgets["properties"].apply(json.loads)

In [6]:
mat2d_titles = [
    s.upper() for s in [
        "Pre-Crisis",
        "Shock/Event",
        "In-Crisis",
        "Sectors",
        "Sectoral Information",
        "Matrix 2D",
    ]
]
mat2d_properties_ids = af_widgets[(
    af_widgets["title"].str.upper()).isin(mat2d_titles)][[
        "properties", "analysis_framework_id"
    ]]
mat2d_properties = mat2d_properties_ids["properties"].tolist()
mat2d_ids = mat2d_properties_ids["analysis_framework_id"].tolist()
mat2d_ids

[552, 537, 726, 799, 273, 829, 495, 1004, 1465, 1465, 1306, 699, 1465]

In [7]:
mat2d_properties

[{'data': {'sectors': [{'id': 'sector-9',
     'title': 'Cross',
     'tooltip': 'Cross sectoral information',
     'subsectors': []},
    {'id': 'sector-0',
     'title': 'Food',
     'tooltip': 'Information related to the food sector',
     'subsectors': []},
    {'id': 'sector-1',
     'title': 'Livelihoods',
     'tooltip': 'Information related to livelihoods, including income, expenses, Minimum Expenditure Basket, access to cash and cash sources, assets and livelihood strategies',
     'subsectors': []},
    {'id': 'sector-2',
     'title': 'Health',
     'tooltip': 'Information related to health, including health system performance, health status of affected people and health resources and services ',
     'subsectors': []},
    {'id': 'sector-3',
     'title': 'Nutrition',
     'tooltip': 'Information related to the nutritional status of the affected population',
     'subsectors': []},
    {'id': 'sector-4',
     'title': 'WASH',
     'tooltip': 'Information related to the WASH

In [8]:
afids_pillars_subpillars = dict()
for mat, af_id in zip(mat2d_properties, mat2d_ids):
    dims = mat["data"]['dimensions']
    afids_pillars_subpillars[af_id] = {}
    for dim in dims:
        pillar = dim["title"]
        sub_pillars = []
        for sub_pillar_dict in dim["subdimensions"]:
            sub_pillars.append(sub_pillar_dict["title"])
        afids_pillars_subpillars[af_id][pillar] = sub_pillars
afids_pillars_subpillars

{552: {'Scope & Scale': ['Drivers/Aggravating Factors',
   'System Disruption',
   'Damages & Losses',
   'Lessons Learnt'],
  'Humanitarian Conditions': ['Living Standards',
   'Coping Mechanisms',
   'Physical & mental wellbeing',
   'Risks & Vulnerabilities',
   'People with Specific Needs',
   'Unmet Needs',
   'Lessons Learnt'],
  'Capacities & Response': ['System Functionality',
   'Government',
   'LNGO',
   'International',
   'Response Gaps',
   'Lessons Learnt']},
 537: {'Scope & Scale': ['Drivers/Aggravating Factors',
   'System Disruption',
   'Damages & Losses',
   'Lessons Learnt'],
  'Humanitarian Conditions': ['Living Standards',
   'Coping Mechanisms',
   'Physical & mental wellbeing',
   'Risks & Vulnerabilities',
   'People with Specific Needs',
   'Unmet Needs',
   'Lessons Learnt'],
  'Capacities & Response': ['System Functionality',
   'Government',
   'LNGO',
   'International',
   'Response Gaps',
   'Lessons Learnt']},
 726: {'Scope & Scale': ['Drivers/Aggravat

In [9]:
af_widgets["title"].tolist()

['Sectoral Information',
 'LOCATION',
 'EXCERPT',
 'Operational Environment',
 'Information Date',
 'Operational Environment',
 'Sectoral Information',
 'LOCATION',
 'EXCERPT',
 'RELIABILITY',
 'Flag',
 'DEMOGRAPHIC GROUPS',
 'SPECIFIC NEEDS GROUPS',
 'AFFECTED GROUPS',
 'Information Date',
 'Operational Environment',
 'Severity',
 'Sectoral Information',
 'LOCATION',
 'Sectors',
 'Cross sector',
 'Affected groups',
 'Specific Needs Groups',
 'Demographic Groups',
 'Reliability',
 'Severity',
 'Geo Location',
 'Excerpt',
 'Information date',
 'Information Date',
 'Sectors',
 'Operational Environment',
 'SPECIFIC NEEDS GROUPS',
 'Sectoral Information',
 'Sectoral Information',
 'Operational Environment',
 'Sectoral Information',
 'LOCATION',
 'Severity',
 'Operational Environment',
 'Information Date',
 'AFFECTED GROUPS',
 'SPECIFIC NEEDS GROUPS',
 'DEMOGRAPHIC GROUPS',
 'Flag',
 'RELIABILITY',
 'EXCERPT',
 'EXCERPT',
 'Severity',
 'Operational Environment',
 'AFFECTED GROUPS',
 'SPECIF

In [10]:
exportables["data"] = exportables["data"].apply(literal_eval)

In [11]:
def extract_title(x):
    if x["excel"].get("title"):
        return x["excel"]["title"]
    elif x["excel"].get("type") == "multiple":
        return x["excel"]["titles"]

af_titles = exportables["data"].apply(extract_title).tolist()

In [12]:
af_titles

['SEVERITY',
 ['Sectoral Information - Dimension',
  'Sectoral Information - Subdimension',
  'Sectoral Information - Sector',
  'Sectoral Information - Subsectors'],
 'DATE OF INFORMATION',
 'RELIABILITY',
 'Severity',
 ['Information Date (From)', 'Information Date (To)'],
 ['AFFECTED GROUPS - Level 0',
  'AFFECTED GROUPS - Level 1',
  'AFFECTED GROUPS - Level 2',
  'AFFECTED GROUPS - Level 3'],
 'Severity',
 ['Information Date (From)', 'Information Date (To)'],
 'SPECIFIC NEEDS GROUPS',
 'DEMOGRAPHIC GROUPS',
 'RELIABILITY',
 ['Sectoral Information - Dimension',
  'Sectoral Information - Subdimension',
  'Sectoral Information - Sector',
  'Sectoral Information - Subsectors'],
 'RELIABILITY',
 ['Flag - Dimension', 'Flag - Subdimension'],
 'DEMOGRAPHIC GROUPS',
 'SPECIFIC NEEDS GROUPS',
 ['Information Date (From)', 'Information Date (To)'],
 ['AFFECTED GROUPS - Level 0',
  'AFFECTED GROUPS - Level 1',
  'AFFECTED GROUPS - Level 2',
  'AFFECTED GROUPS - Level 3'],
 ['Operational Environ

In [13]:
entries.shape, exportdata.shape

((181142, 21), (1018086, 4))

In [14]:
entries[~entries["excerpt"].isna()]["excerpt"].unique().shape

(138924,)

In [15]:
exid_to_exdata = dict()
for ex_id, ex_data in zip(exportables["id"], exportables["data"]):
    exid_to_exdata[ex_id] = ex_data

In [16]:
af_widgets["title"].unique()

array(['Sectoral Information', 'LOCATION', 'EXCERPT',
       'Operational Environment', 'Information Date', 'RELIABILITY',
       'Flag', 'DEMOGRAPHIC GROUPS', 'SPECIFIC NEEDS GROUPS',
       'AFFECTED GROUPS', 'Severity', 'Sectors', 'Cross sector',
       'Affected groups', 'Specific Needs Groups', 'Demographic Groups',
       'Reliability', 'Geo Location', 'Excerpt', 'Information date',
       'PRE-CRISIS', 'IN-CRISIS', 'Crisis type',
       'Context additional tags', 'Crisis Type', 'SHOCK/EVENT',
       'Additional Context', 'DISPLACED POP TYPE', 'CLEANING tags',
       'POPULATION GROUPS', 'Cleaning comments', 'HIGH LEVEL TAGS',
       'Comment'], dtype=object)

In [17]:
widget_key_id_to_title = dict()
for w_key, w_id, title in zip(af_widgets["key"], af_widgets["widget_id"], af_widgets["title"]):
    widget_key_id_to_title[(w_key, w_id)] = title

In [18]:
def exportdata_to_tag_title(row):
    data = json.loads(row[1])
    wkey, wid = None, None
    if data.get("common"):
        wkey = data.get("common").get("widget_key")
        wid = data["common"].get("widget_id")
    if wkey and wid:
        if widget_key_id_to_title.get((wkey, wid)):
            return widget_key_id_to_title.get((wkey, wid))
    if data.get("report") and data["report"].get("other"):
            if len(data["report"]["other"]) == 1 and data["report"]["other"][0].get("title"):
                if data["report"]["other"][0]["title"]:
                    return data["report"]["other"][0]["title"]
    if isinstance(data.get("excel"), list) and len(data["excel"])==1 and \
     data["excel"][0].get("widget_key") and data["excel"][0].get("widget_id"):
        wkey = data["excel"][0]["widget_key"]
        wid = data["excel"][0]["widget_id"]
        if widget_key_id_to_title.get((wkey, wid)):
            return widget_key_id_to_title.get((wkey, wid))
    exportable_id = row[3]
    if exid_to_exdata[exportable_id]["excel"].get("title"):
        return exid_to_exdata[exportable_id]["excel"]["title"]
    elif exid_to_exdata[exportable_id]["excel"].get("type") == "multiple":
        return exid_to_exdata[exportable_id]["excel"]["titles"]
    raise

In [19]:
exportdata["tag_title"] = exportdata.apply(exportdata_to_tag_title, axis=1)

In [20]:
def exportdata_to_tag_value(row):
    data = json.loads(row[1])
    if isinstance(data["excel"], list) and len(data["excel"])==1:
        return data["excel"][0]["value"]
    if data["excel"].get("type") == "lists":
        return data["excel"]["values"]
    if isinstance(data["excel"], dict) and "value" in data["excel"]:
        return data["excel"].get("value")
    if isinstance(data["excel"], dict) and "values" in data["excel"]:
        return data["excel"].get("values")
    elif "values" in data["common"]:
        return data["common"]["values"]
    elif "value" in data["common"]:
        return data["common"]["value"]
    raise

In [21]:
exportdata["tag_value"] = exportdata.apply(exportdata_to_tag_value, axis=1)

In [22]:
def title_case(tag):
    if isinstance(tag, (list, tuple)):
        return tuple([x.title() for x in tag])
    return tag.title()

In [23]:
exportdata["tag_title"] = exportdata["tag_title"].apply(title_case)

In [24]:
exportdata["tag_title"].unique()
# 1. GIMAC --> Pre-Crisis, Shock/Event, In-Crisis
# 2. 2020 Okular --> Sectoral Information
# 3. Okular Analytics Generic --> Sectoral Information
# 4. Rohingya Framework --> Sectors
# 5. IFRC 2018 --> Sectors
# 6. Colombia AF --> Sectoral Information
# 7. Nigeria Situation Analysis (OA) --> Sectoral Information
# 8. Situation Analysis Generic Yemen --> Matrix 2D
# 9. Situation Analysis Generic Libya --> Sectors

array(['Severity', 'Reliability', 'Population Groups', 'Information Date',
       'Location', 'Sectoral Information', 'Demographic Groups', 'Flag',
       'Operational Environment', 'Shock/Event', 'High Level Tags',
       'In-Crisis', 'Specific Needs Groups', 'Comment',
       'Additional Context', 'Sectors', 'Cross Sector', 'Geo Location',
       'Affected Groups', 'Displaced Pop Type', 'Crisis Type',
       'Cleaning Tags', 'Displacement Profile Details',
       ('Affected Groups - Level 0', 'Affected Groups - Level 1', 'Affected Groups - Level 2', 'Affected Groups - Level 3'),
       'Cleaning Comments',
       ('Sectoral Information - Dimension', 'Sectoral Information - Subdimension', 'Sectoral Information - Sector', 'Sectoral Information - Subsectors'),
       ('Information Date (From)', 'Information Date (To)'),
       ('Operational Environment - Dimension', 'Operational Environment - Subdimension'),
       ('In-Crisis - Dimension', 'In-Crisis - Subdimension', 'In-Crisis - Secto

In [25]:
mat2d_titles = [
    s.title() for s in [
        "Pre-Crisis",
        "Shock/Event",
        "In-Crisis",
        "Sectors",
        "Sectoral Information",
        "Matrix 2D",
    ]
]

In [26]:
exportdata_mat2d = exportdata[exportdata["tag_title"].isin(mat2d_titles)]

In [27]:
exportdata_mat2d.shape, exportdata.shape, entries.shape

((149701, 6), (1018086, 6), (181142, 21))

In [28]:
entries.columns

Index(['id', 'created_at', 'modified_at', 'excerpt', 'image_raw',
       'analysis_framework_id', 'created_by_id', 'lead_id', 'modified_by_id',
       'entry_type', 'information_date', 'order', 'client_id', 'project_id',
       'tabular_field_id', 'dropped_excerpt', 'highlight_hidden', 'verified',
       'verification_last_changed_by_id', 'image_id', 'title'],
      dtype='object')

In [29]:
exportdata.columns

Index(['id', 'data', 'entry_id', 'exportable_id', 'tag_title', 'tag_value'], dtype='object')

In [30]:
entries_labeled_mat2d = pd.merge(entries,
                                 exportdata_mat2d,
                                 how="inner",
                                 left_on="id",
                                 right_on="entry_id",
                                 suffixes=('_entry', '_exportdata'))

In [31]:
entries.shape, entries[~entries["excerpt"].isna()]["excerpt"].unique().shape

((181142, 21), (138924,))

In [32]:
entries_labeled_mat2d.shape, entries_labeled_mat2d.columns

((149701, 27),
 Index(['id_entry', 'created_at', 'modified_at', 'excerpt', 'image_raw',
        'analysis_framework_id', 'created_by_id', 'lead_id', 'modified_by_id',
        'entry_type', 'information_date', 'order', 'client_id', 'project_id',
        'tabular_field_id', 'dropped_excerpt', 'highlight_hidden', 'verified',
        'verification_last_changed_by_id', 'image_id', 'title', 'id_exportdata',
        'data', 'entry_id', 'exportable_id', 'tag_title', 'tag_value'],
       dtype='object'))

In [33]:
def mat2d_labels_to_dimension(tag_value):
    if not isinstance(tag_value, list):
        print(tag_value)
        raise
    if len(tag_value):
        dimensions = []
        for t in tag_value:
            if t[0]:
                dimensions.append(t[0].title())
        return list(set(dimensions))
    else:
        return []

def mat2d_labels_to_subdimension(tag_value):
    if not isinstance(tag_value, list):
        print(tag_value)
        raise
    if len(tag_value):
        subdimensions = []
        for t in tag_value:
            if t[1]:
                subdimensions.append(t[1].title())
        return list(set(subdimensions))
    else:
        return []

def mat2d_labels_to_sector(tag_value):
    if not isinstance(tag_value, list):
        print(tag_value)
        raise
    if len(tag_value):
        sectors = []
        for t in tag_value:
            if t[2]:
                sectors.append(t[2].title())
        return list(set(sectors))
    else:
        return []

def mat2d_labels_to_subsector(tag_value):
    if not isinstance(tag_value, list):
        print(tag_value)
        raise
    if len(tag_value):
        subsectors = []
        for t in tag_value:
            if len(t) == 4 and t[3]:
                if isinstance(t[3], list):
                    subsectors.extend([x.title() for x in t[3]])
                else:
                    subsectors.append(t[3].title())
        return list(set(subsectors))
    else:
        return []

In [34]:
entries_labeled_mat2d["Dimension"] = entries_labeled_mat2d["tag_value"].apply(mat2d_labels_to_dimension)
entries_labeled_mat2d["Subdimension"] = entries_labeled_mat2d["tag_value"].apply(mat2d_labels_to_subdimension)
entries_labeled_mat2d["Sector"] = entries_labeled_mat2d["tag_value"].apply(mat2d_labels_to_sector)
entries_labeled_mat2d["Subsector"] = entries_labeled_mat2d["tag_value"].apply(mat2d_labels_to_subsector)

In [35]:
# entries_labeled_mat2d
entries_labeled_mat2d[~entries_labeled_mat2d["excerpt"].isna()]["excerpt"].unique().shape

(112012,)

In [36]:
dims_unique = set()
dims_occurances = list()
for dims in entries_labeled_mat2d["Dimension"]:
    dims_unique.update(dims)
    dims_occurances.extend(dims)
#dims_unique
Counter(dims_occurances).most_common()

[('Humanitarian Conditions', 71079),
 ('Impact', 25481),
 ('Capacities & Response', 15018),
 ('Operational Environment', 10426),
 ('Effects On Population', 7339),
 ('Scope & Scale', 7182),
 ('Response And Capacities', 7148),
 ('Effects Systems And Networks', 5106),
 ('At Risk', 4897),
 ('Humanitarian Situation', 4695),
 ('Context', 4119),
 ('Stated Priorities And Recommendations', 4114),
 ('Priorities', 4055),
 ('Shock Informaton', 3346),
 ('Coping Capacity', 1427),
 ('Capacities And Response', 838),
 ('Scope And Scale', 734)]

In [37]:
secs_unique = set()
secs_occurances = list()
for secs in entries_labeled_mat2d["Sector"]:
    secs_unique.update(secs)
    secs_occurances.extend(secs)
#secs_unique
Counter(secs_occurances).most_common()

[('Health', 42613),
 ('Protection', 30789),
 ('Livelihoods', 19186),
 ('Cross-Sector', 18476),
 ('Food Security', 14711),
 ('Cross', 14014),
 ('Wash', 13448),
 ('Education', 12450),
 ('Shelter', 9275),
 ('Nutrition', 6220),
 ('Food', 3854),
 ('Agriculture', 3183),
 ('Logistics', 2763),
 ('Shelter And Nfis', 2130),
 ('Cross Cutting', 770),
 ('Cccm', 757),
 ('Logistic', 277),
 ('Nfi', 137),
 ('Livelihood', 134)]

In [38]:
entries_labeled_mat2d.columns

Index(['id_entry', 'created_at', 'modified_at', 'excerpt', 'image_raw',
       'analysis_framework_id', 'created_by_id', 'lead_id', 'modified_by_id',
       'entry_type', 'information_date', 'order', 'client_id', 'project_id',
       'tabular_field_id', 'dropped_excerpt', 'highlight_hidden', 'verified',
       'verification_last_changed_by_id', 'image_id', 'title', 'id_exportdata',
       'data', 'entry_id', 'exportable_id', 'tag_title', 'tag_value',
       'Dimension', 'Subdimension', 'Sector', 'Subsector'],
      dtype='object')

In [39]:
final_df = entries_labeled_mat2d[
    entries_labeled_mat2d["entry_type"].eq("excerpt")
    & (~entries_labeled_mat2d["excerpt"].isna())]

In [40]:
def remove_newlines(excerpt):
    if not isinstance(excerpt, str):
        return excerpt
    return re.sub("\s+", " ", excerpt)

final_df['excerpt'] = final_df['excerpt'].apply(remove_newlines)
final_df['dropped_excerpt'] = final_df['dropped_excerpt'].apply(remove_newlines)
##
final_df = final_df.sort_values("verified").reset_index(drop=True).drop_duplicates(subset='excerpt', keep="last")
final_df.shape, final_df.duplicated(subset="excerpt").sum()

<ipython-input-40-d76ddf66f17b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['excerpt'] = final_df['excerpt'].apply(remove_newlines)
<ipython-input-40-d76ddf66f17b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['dropped_excerpt'] = final_df['dropped_excerpt'].apply(remove_newlines)


((111918, 31), 0)

In [41]:
sector_name_mapper = {
    "Agriculture": "Agriculture",
    "Cccm": "",
    "Cross": "Cross",
    "Cross Cutting": "Cross",
    "Cross-Sector": "Cross",
    "Education": "Education",
    "Food": "Food Security",
    "Food Security": "Food Security",
    "Nutrition": "Nutrition",
    "Health": "Health",
    "Livelihood": "Livelihoods",
    "Livelihoods": "Livelihoods",
    "Logistic": "Logistics",
    "Logistics": "Logistics",
    "Protection": "Protection",
    "Shelter": "Shelter",
    "Shelter And Nfis": "Shelter",
    "Nfi": "",
    "Wash": "WASH",
}

In [42]:
af_title_to_id = dict()
for afid, title in zip(afs["id"], afs["title"]):
    af_title_to_id["title"] = afid

In [43]:
af_id_title = afs[["id", "title"]]
af_id_title.columns = ["analysis_framework_id", "Framework Name"]
matching = pd.merge(matching, af_id_title, how="left", left_on="Framework Name", right_on="Framework Name")
matching.columns

Index(['Framework Name', 'Pillar', 'Sub-pillar', 'Final Pillar Name',
       'Final Sub-pillar Name', 'analysis_framework_id'],
      dtype='object')

In [44]:
matching["Pillar"] = matching["Pillar"].apply(lambda x: x.strip().title())
matching["Sub-pillar"] = matching["Sub-pillar"].apply(lambda x: x.strip().title())
matching["Final Pillar Name"] = matching["Final Pillar Name"].apply(lambda x: x.strip().title())
matching["Final Sub-pillar Name"] = matching["Final Sub-pillar Name"].apply(lambda x: x.strip().title())

In [45]:
pillar_name_mapper = dict()
subpillar_name_mapper = dict()
for pillar, fpillar, subpillar, fsubpillar, af_id in zip(
        matching["Pillar"], matching["Final Pillar Name"],
        matching["Sub-pillar"], matching["Final Sub-pillar Name"],
        matching["analysis_framework_id"]):
    pillar_name_mapper[pillar] = fpillar
    subpillar_name_mapper[subpillar] = fsubpillar

In [46]:
matching["Pillar"].unique().tolist() == list(pillar_name_mapper.keys())

True

In [47]:
matching["Sub-pillar"].unique().tolist() == list(subpillar_name_mapper.keys())

True

In [48]:
def sector_mapper(sec):
    if sec == sec:
        sectors =  [sector_name_mapper.get(s, "") for s in sec]
        return [sec for sec in sectors if sec]
    return []
##
def pillar_mapper(dim):
    if dim == dim:
        dim = [pillar_name_mapper.get(d, "") for d in dim]
        return [d for d in dim if d]
    return []
##
def subpillar_mapper(subdim):
    if subdim == subdim:
        subdim = [subpillar_name_mapper.get(d, "") for d in subdim]
        return [d for d in subdim if d]
    return []

In [49]:
final_df["sectors"] = final_df["Sector"].apply(sector_mapper)
final_df["pillars"] = final_df["Dimension"].apply(pillar_mapper)
final_df["subpillars"] = final_df["Subdimension"].apply(subpillar_mapper)

In [50]:
secs_unique = set()
secs_occurances = list()
for secs in final_df["sectors"]:
    secs_unique.update(secs)
    secs_occurances.extend(secs)
#secs_unique
Counter(secs_occurances).most_common()

[('Health', 31358),
 ('Cross', 25403),
 ('Protection', 24199),
 ('Livelihoods', 14410),
 ('Food Security', 13068),
 ('WASH', 9899),
 ('Education', 9287),
 ('Shelter', 8523),
 ('Nutrition', 4542),
 ('Agriculture', 2593),
 ('Logistics', 2263)]

In [51]:
dims_unique = set()
dims_occurances = list()
for dims in final_df["pillars"]:
    dims_unique.update(dims)
    dims_occurances.extend(dims)
#dims_unique
Counter(dims_occurances).most_common()

[('People At Risk', 53794), ('Impact', 38775), ('Priority Needs', 8839)]

In [52]:
subdims_unique = set()
subdims_occurances = list()
for subdims in final_df["subpillars"]:
    subdims_unique.update(subdims)
    subdims_occurances.extend(subdims)
Counter(subdims_occurances).most_common()

[('Living Standards', 22672),
 ('Physical And Mental Well Being', 13497),
 ('Driver/Aggravating Factors', 11715),
 ('Risk And Vulnerabilities', 9610),
 ('Impact On Systems And Services', 6982),
 ('National Response', 6670),
 ('Impact On People', 6333),
 ('International Response', 4746),
 ('Impact On Services', 4490),
 ('Coping Mechanisms', 3628),
 ('Expressed By Humanitarian Staff', 2775),
 ('Number Of People In Need', 2643),
 ('Impact On People Or Impact On Services', 2550),
 ('Number Of People Affected', 1962),
 ('Number Of People Reached', 1836),
 ('Expressed By Population', 1420),
 ('Response Gaps', 333),
 ('Number Of People At Risk', 277)]

In [53]:
# add columns for sectors
for sector in secs_unique:
    final_df[sector] = 0
# add columns for dimensions
for dimension in dims_unique:
    final_df[dimension] = 0
# add columns for subdimensions
for subdimension in subdims_unique:
    final_df[subdimension] = 0

In [54]:
final_df.columns

Index(['id_entry', 'created_at', 'modified_at', 'excerpt', 'image_raw',
       'analysis_framework_id', 'created_by_id', 'lead_id', 'modified_by_id',
       'entry_type', 'information_date', 'order', 'client_id', 'project_id',
       'tabular_field_id', 'dropped_excerpt', 'highlight_hidden', 'verified',
       'verification_last_changed_by_id', 'image_id', 'title', 'id_exportdata',
       'data', 'entry_id', 'exportable_id', 'tag_title', 'tag_value',
       'Dimension', 'Subdimension', 'Sector', 'Subsector', 'sectors',
       'pillars', 'subpillars', 'Nutrition', 'Logistics', 'Cross',
       'Agriculture', 'WASH', 'Health', 'Food Security', 'Education',
       'Livelihoods', 'Shelter', 'Protection', 'Priority Needs', 'Impact',
       'People At Risk', 'Impact On Systems And Services',
       'Impact On Services', 'Number Of People Affected', 'National Response',
       'Number Of People At Risk', 'International Response',
       'Number Of People In Need', 'Impact On People Or Impact O

In [55]:
for row in tqdm(final_df.iterrows(), total=final_df.shape[0]):
    sectors_i = row[1]["sectors"]
    pillars_i = row[1]["pillars"]
    subpillars_i = row[1]["subpillars"]
    idx = row[0]
    for sector in sectors_i:
        final_df.loc[idx, sector] = 1
    for pillar in pillars_i:
        final_df.loc[idx, pillar] = 1
    for subpillar in subpillars_i:
        final_df.loc[idx, subpillar] = 1

100%|██████████| 111918/111918 [02:01<00:00, 918.31it/s] 


In [56]:
final_df.columns

Index(['id_entry', 'created_at', 'modified_at', 'excerpt', 'image_raw',
       'analysis_framework_id', 'created_by_id', 'lead_id', 'modified_by_id',
       'entry_type', 'information_date', 'order', 'client_id', 'project_id',
       'tabular_field_id', 'dropped_excerpt', 'highlight_hidden', 'verified',
       'verification_last_changed_by_id', 'image_id', 'title', 'id_exportdata',
       'data', 'entry_id', 'exportable_id', 'tag_title', 'tag_value',
       'Dimension', 'Subdimension', 'Sector', 'Subsector', 'sectors',
       'pillars', 'subpillars', 'Nutrition', 'Logistics', 'Cross',
       'Agriculture', 'WASH', 'Health', 'Food Security', 'Education',
       'Livelihoods', 'Shelter', 'Protection', 'Priority Needs', 'Impact',
       'People At Risk', 'Impact On Systems And Services',
       'Impact On Services', 'Number Of People Affected', 'National Response',
       'Number Of People At Risk', 'International Response',
       'Number Of People In Need', 'Impact On People Or Impact O

In [57]:
len(list(secs_unique)+list(dims_unique)+list(subdims_unique))

32

In [58]:
final_df[list(secs_unique)+list(dims_unique)+list(subdims_unique)].sum().astype(int)

Nutrition                                  4542
Logistics                                  2263
Cross                                     25403
Agriculture                                2593
WASH                                       9899
Health                                    31358
Food Security                             13068
Education                                  9287
Livelihoods                               14410
Shelter                                    8523
Protection                                24199
Priority Needs                             8839
Impact                                    37821
People At Risk                            51837
Impact On Systems And Services             6982
Impact On Services                         4417
Number Of People Affected                  1962
National Response                          6339
Number Of People At Risk                    277
International Response                     4746
Number Of People In Need                

### Stratified Split

In [59]:
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix

In [60]:
def train_val_test_df_split(df):
    classes = list(secs_unique)+list(dims_unique)+list(subdims_unique)
    class_to_id = {clss:i for i, clss in enumerate(classes)}
    num_classes = len(classes)
    labels = np.zeros([len(df), num_classes])
    for i, (sectors_i, pillars_i, subpillars_i) in enumerate(zip(df["sectors"], df["pillars"], df["subpillars"])):
        for sec in sectors_i:
            labels[i, class_to_id[sec]] = 1
        for pil in pillars_i:
            labels[i, class_to_id[pil]] = 1
        for subpil in subpillars_i:
            labels[i, class_to_id[subpil]] = 1
    ##
    X_train, y_train, X_test, y_test = iterative_train_test_split(
        df["excerpt"].to_numpy().reshape(-1, 1), labels, test_size=0.1)
    X_train, y_train, X_val, y_val = iterative_train_test_split(X_train,
                                                                y_train,
                                                                test_size=0.1)
    return {
        "X_train": X_train,
        "y_train": y_train,
        "X_test": X_test,
        "y_test": y_test,
        "X_val": X_val,
        "y_val": y_val
    }

In [61]:
df = final_df[[
    'entry_id',
    'lead_id',
    'project_id',
    'title',
    'analysis_framework_id',
    'excerpt',
    'dropped_excerpt',
    'created_by_id',
    'modified_by_id',
    'verified',
    'verification_last_changed_by_id',
    'sectors',
    'pillars',
    'subpillars',
]]
df.columns = [
    'entry_id',
    'lead_id',
    'project_id',
    'project_title',
    'analysis_framework_id',
    'excerpt',
    'dropped_excerpt',
    'created_by_id',
    'modified_by_id',
    'verified',
    'verification_last_changed_by_id',
    'sectors',
    'pillars',
    'subpillars',
]

In [62]:
X_train, y_train, X_test, y_test, X_val, y_val = list(
    train_val_test_df_split(df).values())

In [63]:
final_df_train = final_df[final_df["excerpt"].isin(X_train.reshape(-1))]
final_df_val = final_df[final_df["excerpt"].isin(X_val.reshape(-1))]
final_df_test = final_df[final_df["excerpt"].isin(X_test.reshape(-1))]

In [64]:
df_train = df[df["excerpt"].isin(X_train.reshape(-1))]
df_val = df[df["excerpt"].isin(X_val.reshape(-1))]
df_test = df[df["excerpt"].isin(X_test.reshape(-1))]

In [65]:
df_train.index & df_val.index, df_train.index & df_test.index, df_test.index & df_val.index 

(Int64Index([], dtype='int64'),
 Int64Index([], dtype='int64'),
 Int64Index([], dtype='int64'))

In [66]:
df_train.to_csv("data_v0.4.2_train.csv", index=None)
df_val.to_csv("data_v0.4.2_val.csv", index=None)
df_test.to_csv("data_v0.4.2_test.csv", index=None)

In [67]:
df.to_csv("data_v0.4.2_full.csv", index=None)